# Experiment 4.4

## Includes

In [1]:
using CSV
using DataFrames
using Random
using Glob

In [2]:
using Gadfly
import Cairo, Fontconfig

In [3]:
ENV["COLUMNS"] = 1000;

## Load Data

In [4]:
fns_a = glob("*.csv", "../results/experiment_4_4a/")
dfs_a = [DataFrame(CSV.File(fn)) for fn in fns_a]
results_exp44a = vcat(dfs_a...);

In [5]:
fns_b = glob("*.csv", "../results/experiment_4_4b/")
dfs_b = [DataFrame(CSV.File(fn)) for fn in fns_b]
results_exp44b = vcat(dfs_b...);

In [6]:
fns_c = glob("*.csv", "../results/experiment_4_4c/")
dfs_c = [DataFrame(CSV.File(fn)) for fn in fns_c]
results_exp44c = vcat(dfs_c...);

In [7]:
function get_match_ratio_colorid(x)
    if x == 1.0
        return 4
    elseif x >= 0.85
        return 3
    elseif x >= 0.5
        return 2
    else
        return 1
    end
end
results_exp44a.match_color = map(get_match_ratio_colorid, results_exp44a.match_ratio);
results_exp44b.match_color = map(get_match_ratio_colorid, results_exp44b.match_ratio);
results_exp44c.match_color = map(get_match_ratio_colorid, results_exp44c.match_ratio);

In [8]:
@show nrow(results_exp44a)
first(results_exp44a, 2)

nrow(results_exp44a) = 41


,N,n,m,μ,σ,ρe,ρh,ρt,match_ratio,alignment_strength_1,alignment_strength_2,iter,maxiter,sgm_time,distribution,experiment,match_color
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,String,String,Int64
1,1250,1000,250,0.459632,0.0580002,0.0,0.0135444,0.0135444,0.0,0.113266,0.116096,20,20,23.8854,bernoulli,44a,1
2,1250,1000,250,0.459632,0.0580002,0.025,0.0135444,0.0382058,0.003,0.110931,0.116241,20,20,25.3542,bernoulli,44a,1


In [9]:
@show nrow(results_exp44b)
first(results_exp44b, 2)

nrow(results_exp44b) = 402


,N,n,m,μ,σ,ρe,ρh,ρt,match_ratio,alignment_strength_1,alignment_strength_2,iter,maxiter,sgm_time,distribution,experiment,match_color
,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,String,String,Int64
1,1040,1000,40,0.46029,0.238305,0.0,0.228598,0.228598,0.046,0.124282,0.131522,20,20,27.2943,bernoulli,44b,1
2,1040,1000,40,0.459487,0.237651,0.005,0.227404,0.231267,0.055,0.126097,0.132387,20,20,25.4501,bernoulli,44b,1


In [10]:
@show nrow(results_exp44c)
first(results_exp44c, 2)

nrow(results_exp44c) = 328


,N,n,m,μ,dist_idx,ρe,ρh,ρt,σ,ρh_mod,ρt_mod,σ2_mod,match_ratio,alignment_strength_1,alignment_strength_2,iter,maxiter,sgm_time,distribution,experiment,match_color
,Int64,Int64,Int64,Float64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Float64,String,String,Int64
1,1250,1000,250,0.459615,1,0.0,0.0169267,0.0169267,0.0648389,0.00335373,0.00335373,0.000832963,0.001,0.111601,0.116007,20,20,27.4763,bernoulli,44c,1
2,1250,1000,250,0.459617,2,0.0,0.0215752,0.0215752,0.0732025,0.00803603,0.00803603,0.0019959,0.005,0.112306,0.117157,20,20,20.5214,bernoulli,44c,1


## Style

In [11]:
defaultfont = "CMU Serif";
fontstyles = (
    key_title_font = defaultfont,
    key_label_font = defaultfont,
    minor_label_font = defaultfont,
    major_label_font = defaultfont,
    key_title_font_size = 14px,
    key_label_font_size = 12px,
    minor_label_font_size = 14px,
    major_label_font_size = 16px,
);

In [12]:
colors = [colorant"red", colorant"darkblue", colorant"deepskyblue", colorant"green"]

## Figures

In [13]:
figurespath = "./figures/tex/"
if !isdir(figurespath) mkpath(figurespath) end;

In [14]:
fig1 = plot(
    results_exp44a,
    x = :ρt, y = :alignment_strength_1, color = :match_color,
    Guide.xlabel("Total Correlation (ρ<sub>T</sub>)"),
    Guide.ylabel("Alignment Strength"),
    Guide.xticks(ticks=0:0.25:1),
    Guide.yticks(ticks=0:0.25:1),
    Scale.color_discrete_manual(colors..., levels=[1,2,3,4]),
    Guide.colorkey(title="Match Ratio", labels=["&lt;50%", "≥50%", "≥85%", "100%"]),
    style(
        ;fontstyles...,
    ),
)
fig1 |> SVG(16cm, 10cm)
fig1 |> PGF(joinpath(figurespath, "exp44_fig1.tex"), 16cm, 10cm, texfonts=true);

In [15]:
fig2 = plot(
    results_exp44b,
    x = :ρt, y = :alignment_strength_1, color = :match_color,
    Guide.xlabel("Total Correlation (ρ<sub>T</sub>)"),
    Guide.ylabel("Alignment Strength"),
    Guide.xticks(ticks=0:0.25:1),
    Guide.yticks(ticks=0:0.25:1),
    Scale.color_discrete_manual(colors..., levels=[1,2,3,4]),
    Guide.colorkey(title="Match Ratio", labels=["&lt;50%", "≥50%", "≥85%", "100%"]),
    style(
        highlight_width=0px,
        point_size=0.6mm,
        ;fontstyles...,
    ),
)
fig2 |> SVG(16cm, 10cm)
fig2 |> PGF(joinpath(figurespath, "exp44_fig2.tex"), 16cm, 10cm, texfonts=true);

In [16]:
fig3 = plot(
    results_exp44c,
    x = :ρt, y = :alignment_strength_1, color = :match_color,
    Guide.xlabel("Total Correlation (ρ<sub>T</sub>)"),
    Guide.ylabel("Alignment Strength"),
    Guide.xticks(ticks=0:0.25:1),
    Guide.yticks(ticks=0:0.25:1),
    Scale.color_discrete_manual(colors..., levels=[1,2,3,4]),
    Guide.colorkey(title="Match Ratio", labels=["&lt;50%", "≥50%", "≥85%", "100%"]),
    style(
        highlight_width=0.5px,
        point_size=0.5mm,
        ;fontstyles...,
    ),
)
fig3 |> SVG(16cm, 10cm)
fig3 |> PGF(joinpath(figurespath, "exp44_fig3.tex"), 16cm, 10cm, texfonts=true);

In [17]:
for dist_idx in unique(results_exp44c.dist_idx)
    _results = filter(row -> row.dist_idx == dist_idx, results_exp44c)
    fig4 = plot(
        _results,
        x = :ρt, y = :alignment_strength_1, color = :match_color,
        Guide.xlabel("Total Correlation (ρ<sub>T</sub>)"),
        Guide.ylabel("Alignment Strength"),
        Guide.title("($(dist_idx))"),
        Guide.xticks(ticks=0:0.25:1),
        Guide.yticks(ticks=0:0.25:1),
        Scale.color_discrete_manual(colors..., levels=[1,2,3,4]),
        Guide.colorkey(title="Match Ratio", labels=["&lt;50%", "≥50%", "≥85%", "100%"]),
        style(
            highlight_width=0.5px,
            point_size=0.5mm,
            ;fontstyles...,
        ),
    )
    fig4 |> SVG(16cm, 10cm)
    fig4 |> PGF(joinpath(figurespath, "exp44_fig4_d$(dist_idx).tex"), 16cm, 10cm, texfonts=true);
end